In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:],scene['v_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])     
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:],scene['v_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 4,
                 embedding_size = 16,
                 hidden_size = 1792):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=16, hidden_size=1792, output_size=4):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.linear1(x)
        embedded = F.relu(x)
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(iterator):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #output whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :]
        
        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden
#         print("DECODER INPUT", decoder_input.shape)
#         print("DECODER HIDDEN", decoder_hidden[0].shape)
        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :4], target[:, di, :4]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        if counter % (0.1*len(train_loader)) == 0:
            print("10% passed")
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=4)
decoder = DecoderRNN(output_size=4)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 2
gamma=0.1

for epoch in range(1, num_epoch + 1):
    for param_group in encoder_optimizer.param_groups:
        if epoch == 2:
            param_group['lr'] *= gamma
    for param_group in decoder_optimizer.param_groups:
        if epoch == 2:
            param_group['lr'] *= gamma
            
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :4]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 4)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
1890,"[253.1996307373047, 812.2052612304688, 7.05087...","[253.8334197998047, 812.280029296875, 7.066197...","[254.6178741455078, 812.3115234375, 7.14329242...","[255.40858459472656, 812.4013061523438, 7.1798...","[256.1741638183594, 812.3433227539062, 7.23062...","[256.8155517578125, 812.3859252929688, 7.20263...","[257.49871826171875, 812.4632568359375, 7.1424...","[258.2076416015625, 812.46142578125, 7.2880558...","[258.9261779785156, 812.399169921875, 7.399316...","[259.67938232421875, 812.5086059570312, 7.3925...",...,"[267.94708251953125, 813.041259765625, 7.45035...","[268.78729248046875, 813.0277099609375, 7.5119...","[269.6463317871094, 813.1300659179688, 7.49400...","[270.159423828125, 813.1895141601562, 7.407918...","[271.02069091796875, 813.1295776367188, 7.4042...","[271.71478271484375, 813.2415161132812, 7.4054...","[272.4834289550781, 813.324951171875, 7.328201...","[273.3404846191406, 813.4774169921875, 7.21407...","[273.8618469238281, 813.1076049804688, 7.15482...","[274.4111328125, 813.15576171875, 7.0662946701..."
14703,"[1902.5224609375, 494.1365966796875, 10.706312...","[1903.6431884765625, 495.03997802734375, 10.41...","[1904.545166015625, 495.86297607421875, 10.254...","[1905.6846923828125, 496.55230712890625, 10.25...","[1906.6884765625, 497.48443603515625, 10.23212...","[1907.780029296875, 498.43560791015625, 10.291...","[1908.8145751953125, 499.2645568847656, 10.230...","[1909.8485107421875, 499.981201171875, 10.3184...","[1910.818603515625, 500.96270751953125, 10.374...","[1911.9554443359375, 501.83111572265625, 10.33...",...,"[1923.20068359375, 511.74652099609375, 10.2883...","[1924.18798828125, 512.5670776367188, 10.30213...","[1925.3240966796875, 513.5682373046875, 10.288...","[1926.294921875, 514.3320922851562, 10.3551836...","[1927.3095703125, 515.3047485351562, 10.302641...","[1928.2576904296875, 516.0178833007812, 10.223...","[1929.2342529296875, 516.8807373046875, 10.241...","[1930.3388671875, 517.898681640625, 10.1705379...","[1931.3101806640625, 518.7006225585938, 10.068...","[1932.2470703125, 519.3699340820312, 9.9695091..."
6049,"[1987.21728515625, 572.1846313476562, -9.80676...","[1986.20556640625, 571.6297607421875, -10.1223...","[1985.199462890625, 570.777099609375, -10.1062...","[1984.111328125, 570.0983276367188, -10.094079...","[1983.050048828125, 569.3920288085938, -10.135...","[1982.0096435546875, 568.5689697265625, -10.12...","[1980.8753662109375, 567.8448486328125, -10.21...","[1979.924072265625, 566.9252319335938, -10.220...","[1979.0211181640625, 565.9296264648438, -10.12...","[1977.9510498046875, 565.1686401367188, -10.15...",...,"[1966.7540283203125, 556.4508666992188, -10.12...","[1965.687744140625, 555.6363525390625, -10.071...","[1964.7061767578125, 554.8351440429688, -10.06...","[1963.8402099609375, 553.9574584960938, -10.08...","[1962.80517578125, 553.3634033203125, -10.1396...","[1961.7755126953125, 552.5415649414062, -10.04...","[1960.69970703125, 551.6488037109375, -10.0983...","[1959.8779296875, 550.761474609375, -10.004471...","[1958.77685546875, 550.0362548828125, -9.88512...","[1957.83447265625, 549.0172729492188, -9.77489..."
16850,"[739.107421875, 1486.1087646484375, -0.2085859...","[738.60498046875, 1487.9410400390625, -0.82415...","[738.7239990234375, 1489.83203125, -0.71436369...","[738.5366821289062, 1491.56396484375, -0.78136...","[738.4448852539062, 1493.2919921875, -0.795238...","[738.4535522460938, 1495.000244140625, -0.6531...","[738.1688842773438, 1496.673583984375, -0.7718...","[738.196044921875, 1498.57763671875, -0.713045...","[738.0657958984375, 1500.255859375, -0.6843398...","[738.050537109375, 1501.688720703125, -0.70400...",...,"[737.4175415039062, 1520.248291015625, -0.6317...","[737.35888671875, 1521.9495849609375, -0.49009...","[737.3118896484375, 1523.609619140625, -0.5063...","[737.4002075195312, 1525.0975341796875, -0.509...","[737.2173461914062, 1527.017822265625, -0.4597...","[737.1

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)[[0, 1]]

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
1890,253.199631,812.205261,253.833420,812.280029,254.617874,812.311523,255.408585,812.401306,256.174164,812.343323,...,271.714783,813.241516,272.483429,813.324951,273.340485,813.477417,273.861847,813.107605,274.411133,813.155762
14703,1902.522461,494.136597,1903.643188,495.039978,1904.545166,495.862976,1905.684692,496.552307,1906.688477,497.484436,...,1928.257690,516.017883,1929.234253,516.880737,1930.338867,517.898682,1931.310181,518.700623,1932.247070,519.369934
6049,1987.217285,572.184631,1986.205566,571.629761,1985.199463,570.777100,1984.111328,570.098328,1983.050049,569.392029,...,1961.775513,552.541565,1960.699707,551.648804,1959.877930,550.761475,1958.776855,550.036255,1957.834473,549.017273
16850,739.107422,1486.108765,738.604980,1487.941040,738.723999,1489.832031,738.536682,1491.563965,738.444885,1493.291992,...,737.186951,1528.291382,737.115295,1529.909790,736.876160,1531.345703,736.987854,1533.326416,737.133545,1534.596436
37523,1708.117432,338.751038,1707.291382,338.233032,1706.510010,337.502808,1705.666138,336.826660,1704.935669,336.217133,...,1688.116943,321.795013,1687.285400,321.067963,1686.497925,320.388336,1685.809570,319.619812,1684.966431,318.988464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31834,1738.669922,454.411469,1738.747681,453.797882,1738.675049,453.541779,1738.849854,453.042236,1738.893677,452.467285,...,1742.248413,442.904053,1742.640991,442.421967,1742.958374,441.945496,1743.129272,441.565063,1743.065552,441.215576
11962,1787.582642,408.851807,1786.796021,408.121429,1786.080200,407.530273,1785.263794,406.797455,1784.533569,406.195831,...,1768.057983,392.003937,1767.240601,391.192230,1766.388306,390.436310,1765.743408,389.702698,1764.950073,389.201324
18370,1795.253906,408.165588,1796.126099,409.071655,1796.997803,409.833771,1798.053955,410.502594,1798.988647,411.329102,...,1818.516113,428.057495,1819.349854,428.735626,1820.438599,429.816742,1821.358276,430.540466,1822.203857,431.185455


In [12]:
df2.to_csv("outputs2epb4_v_hidden1792_all_gamma1.csv", index=True)